## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-26-17-46-15 +0000,nyt,As the Left’s Biggest Stars Unite Behind Mamda...,https://www.nytimes.com/2025/10/26/nyregion/ma...
1,2025-10-26-17-43-45 +0000,bbc,Rebel group claims capture of key city in Sudan,https://www.bbc.com/news/articles/c78z025dze4o...
2,2025-10-26-17-43-06 +0000,nyt,Newsom Says He Will Consider a 2028 Presidenti...,https://www.nytimes.com/2025/10/26/us/politics...
3,2025-10-26-17-41-00 +0000,wsj,Boeing Defense Workers Reject Latest Contract,https://www.wsj.com/business/boeing-defense-wo...
4,2025-10-26-17-39-07 +0000,nyt,President Oversees Signing of Cambodia-Thailan...,https://www.nytimes.com/live/2025/10/25/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,41
108,deal,10
307,louvre,10
37,melissa,9
359,heist,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2025-10-26-16-00-33 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...,79
118,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...,78
50,2025-10-26-13-39-19 +0000,bbc,Will Trump's 'peace deal' solve Thailand-Cambo...,https://www.bbc.com/news/videos/cy40n3ykx93o?a...,76
22,2025-10-26-16-10-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...,76
176,2025-10-25-20-30-41 +0000,nypost,Trump not meeting with Putin again until he kn...,https://nypost.com/2025/10/25/world-news/trump...,75


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,79,2025-10-26-16-00-33 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...
95,51,2025-10-26-09-47-45 +0000,nypost,2 Louvre robbery suspects busted in $100M muse...,https://nypost.com/2025/10/26/world-news/louvr...
106,40,2025-10-26-09-00-00 +0000,wsj,Rep. Jared Golden didn’t just side with the GO...,https://www.wsj.com/politics/policy/jared-gold...
22,35,2025-10-26-16-10-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...
117,31,2025-10-26-07-02-02 +0000,nypost,Hurricane Melissa strengthens into Category 3 ...,https://nypost.com/2025/10/26/world-news/hurri...
28,29,2025-10-26-15-53-01 +0000,nypost,Jeffrey Epstein victim Virginia Giuffre feared...,https://nypost.com/2025/10/26/us-news/virginia...
19,28,2025-10-26-16-16-26 +0000,latimes,Police arrest two suspects in fatal shooting o...,https://www.latimes.com/california/story/2025-...
118,28,2025-10-26-06-49-24 +0000,nypost,"Trump signs peace, trade deals ending conflict...",https://nypost.com/2025/10/26/us-news/trump-si...
18,25,2025-10-26-16-33-14 +0000,cbc,Russia has tested a new nuclear-capable missil...,https://www.cbc.ca/news/world/russia-nuclear-c...
136,25,2025-10-26-01-46-12 +0000,nypost,Michigan dad allegedly threatened to shoot up ...,https://nypost.com/2025/10/25/us-news/michigan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
